In [2]:
import fastf1

# Enabling the caching of data, so it doesn't need to get downloaded every time
fastf1.Cache.enable_cache('cache')

In [3]:
session = fastf1.get_session(2024, 'Saudi Arabia', 'Q')
session.load()

core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req           

In [6]:
ver_lap = session.laps.pick_drivers('VER').pick_fastest()

In [7]:
telemetry = ver_lap.get_telemetry()

In [8]:
print(telemetry[['Speed', 'RPM', 'Throttle', 'Brake']].head())

        Speed           RPM  Throttle  Brake
2  313.930000  11144.470000     100.0  False
3  314.207500  11147.892494     100.0  False
4  316.000000  11170.000000     100.0  False
5  316.150001  11177.200066     100.0  False
6  317.292859  11232.057209     100.0  False


In [9]:
print(f"\nTop Speed: {telemetry['Speed'].max()} km/h")


Top Speed: 331.0 km/h
